In [148]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
# Import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [149]:
def extract_info(url = 'https://www.todayonline.com/', name = 'today'):
    
    driver = webdriver.Chrome(executable_path = r'C:\Users\65961\OneDrive\Desktop\Data_Products\chromedriver.exe')
    driver.get(url)
    delay = 5
    try:
        myElem = WebDriverWait(driver, delay)
        print("Page {} is ready!".format(driver.current_url))
    except TimeoutException:
        print("Loading took too much time!")
    current_page_source = driver.page_source
    soup = BeautifulSoup(current_page_source, 'html.parser')
    
    # Extract info
    hotnews = soup.find_all('section', {'categoryid':'7'})
    #hotnews_list = hotnews[0].text.split(sep = 'ago')
    hotnews_soup = hotnews[0].find_all('a')
    hotnews_list = []
    
    for item in hotnews_soup:
        hotnews_dict = {'Title': item.text,
                       'Type': 'hotnews',
                        'URL': url,
                        'HREF': item['href'],
                        'Name': name,
                       'Date': datetime.datetime.now().strftime('%Y-%m-%d'),
                       'Datetime': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        hotnews_list.append(hotnews_dict)
    
    popularnews = soup.find_all('section', {'categoryid':'3'})
    #popularnews_list = popularnews[0].text.split(sep = 'ago')
    popularnews_soup = popularnews[0].find_all('a')
    popularnews_list = []
    
    for item in popularnews_soup:
        popularnews_dict = {'Title': item.text,
                       'Type': 'popularnews',
                        'URL': url,
                        'HREF': item['href'],
                        'Name': name,
                       'Date': datetime.datetime.now().strftime('%Y-%m-%d'),
                       'Datetime': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        popularnews_list.append(popularnews_dict)
    
    worldnews = soup.find_all('section', {'categoryid':'5'})
    #worldnews_list = worldnews[0].text.split(sep = 'ago')
    worldnews_soup = worldnews[0].find_all('a')
    worldnews_list = []
    
    for item in worldnews_soup:
        worldnews_dict = {'Title': item.text,
                       'Type': 'worldnews',
                        'URL': url,
                        'HREF': item['href'],
                        'Name': name,
                       'Date': datetime.datetime.now().strftime('%Y-%m-%d'),
                       'Datetime': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        worldnews_list.append(worldnews_dict)
    

    
    mega_list = [hotnews_list, popularnews_list, worldnews_list]
    
    #mega_list2 = []

    #i = 0

    #for item in mega_list:
    #    if i == 0:
    #        news_type = 'hotnews'
    #    elif i == 1:
    #        news_type = 'popularnews'
    #    elif i == 2:
    #        news_type = 'worldnews'
    #    for item2 in item:
    #        item2_dict = {'News': item2,
    #                     'News_Original': item2 + ' ago',
    #                      'News_Type': news_type,
    #                      'URL': url,
    #                      'Name': name,
    #                     'Date': datetime.datetime.now().strftime('%Y-%m-%d'),
    #                     'Datetime': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    #        mega_list2.append(item2_dict)
    #    i = i + 1

    mega_df = pd.DataFrame(mega_list)
    
    mega_df = pd.concat([pd.DataFrame(hotnews_list), pd.DataFrame(popularnews_list), pd.DataFrame(worldnews_list)], axis = 0).reset_index(drop = True)

    mega_df = mega_df.drop_duplicates(subset = ['HREF'], keep = 'last').reset_index(drop = True)
    
    mega_df.to_csv('C:\\Users\\65961\\OneDrive\\Desktop\\Data_Products\\' + '{}_news_scraper_v3_'.format(name) + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.csv', index = False)
    
    return mega_df

In [150]:
def extract_article(df):
    
    href_list = df['HREF']
    url_list = df['URL']

    link_list = []
    for item in range(0, df.shape[0]):
        link_list.append(url_list[item][:-1] + href_list[item])
        
    df2 = df
    df2['Link'] = link_list
        
    article_list = []    
    for item in link_list:
        time.sleep(3)
        url = item
        driver = webdriver.Chrome(executable_path = r'C:\Users\65961\OneDrive\Desktop\Data_Products\chromedriver.exe')
        driver.get(url)
        delay = 5
        try:
            myElem = WebDriverWait(driver, delay)
            print("Page {} is ready!".format(driver.current_url))
        except TimeoutException:
            print("Loading took too much time!")
        
        current_page_source = driver.page_source
        soup = BeautifulSoup(current_page_source, 'html.parser')
        
        article = soup.find_all('div', {'id': 'article_detail_body'})
        
        if len(article) < 1:
            continue
        
        article2 = article[0].find_all('p')
        
        article3 = ''
        for item2 in article2:
            article3 = article3 + ' ' + item2.text
        article4 = article3.replace('\n', '').replace('\xa0', ' ').strip()
        
        article_dict = {'Link': item,
                       'Article': article4}
        article_list.append(article_dict)
        
    #article_df = pd.DataFrame(article_list)
    
    #df2 = pd.merge(df2, article_df, how = 'left', on = ['Link'])
    
    #df2 = df2.reset_index(drop = True)
    
    #df2_name = df2['Name']
    
    #name = df2_name[0]
    
    #df2.to_csv('C:\\Users\\65961\\OneDrive\\Desktop\\Data_Products\\' + '{}_news_scraper_article_v3_'.format(name) + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.csv', index = False)
    
    return df2, article_list
        


In [151]:
def merge_data(df, list_):
    
    list_df = pd.DataFrame(list_)
    
    df = pd.merge(df, list_df, how = 'left', on = ['Link'])
    
    df = df.reset_index(drop = True)
    
    df_name = df['Name']
    
    name = df_name[0]
    
    df.to_csv('C:\\Users\\65961\\OneDrive\\Desktop\\Data_Products\\' + '{}_news_scraper_article_v3_'.format(name) + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.csv', index = False)
    
    return df
    

In [152]:
mega_df = extract_info()

Page https://www.todayonline.com/ is ready!


In [153]:
mega_df2, article_list = extract_article(mega_df)

C:\Users\65961\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Page https://www.todayonline.com/singapore/new-covid-19-cases-dec-26 is ready!
Page https://www.todayonline.com/singapore/covid-19-not-likely-be-eradicated-sars-ncid-chief-says is ready!
Page https://www.todayonline.com/singapore/moh-orders-concord-international-hospital-suspend-healthcare-services-due-significant is ready!


In [154]:
mega_df3 = merge_data(mega_df2, article_list)